In [1]:
from main import TranscriptScraper

In [2]:
scraper = TranscriptScraper(start_date='2024-06-06', end_date='2024-06-06')

In [3]:
scraper.build_and_save_transcripts()

Home-TV-en-3850
yt-dlp -f Home-TV-en-3850 --write-subs --sub-lang en --embed-subs -P ../../../data/raw_transcripts --username jaredzrks0@gmail.com --password Emiliopagan450 --skip-download https://www.mlb.com/tv/g744923/vc993d634-cb00-42b6-9a2d-5beafe96e808


KeyboardInterrupt: 

In [4]:
scraper.game_urls

[]

In [ ]:
from selenium import webdriver
import chromedriver_autoinstaller
import subprocess
import json


chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path

driver = webdriver.Chrome()
driver.get("http://www.python.org")
assert "Python" in driver.title

In [ ]:
import pandas as pd
import configparser
import requests

from bs4 import BeautifulSoup


# Load the config file
config = configparser.ConfigParser()
config.read('../../../config.ini')

# Access credentials
username = config['credentials']['username']
password = config['credentials']['password']


## Scrape all the urls from mlb.com where games are listed

On MLB.com, there is a webpage each day that lists all of the games that are played, in addition to the links to both the home and away broadcast. These links are where we will get the keywords needed to run the command line transcript downloader.

In [18]:
# Define the base url for the webpage that lists all of the day's games with links to the home and away video
base_url = 'https://www.mlb.com/live-stream-games/'

# Create a list of dates that are included in our pull (3 years are available)
dates = pd.date_range('2022-04-01', '2024-11-10')
suffixes = [date.strftime("%Y/%m/%d") for date in dates]

# Define the total list of URLs for the webpages with the day's games. Note this includes offseason days too
total_day_urls = [base_url + suffix for suffix in suffixes]

### Grab the game links from each day's URL ###

# for url in total_day_urls:
#     request = requests.get(url)
#     soup = BeautifulSoup(request.text, 'html.parser')

    

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Set up Selenium
driver = webdriver.Chrome()

# Open the MLB page
url = "https://www.mlb.com/live-stream-games/2023/06/14"
driver.get(url)

# Wait for elements to load (adjust timeout as needed)
try:
    # Wait until the element is clickable (replace with the correct selector)
    button = WebDriverWait(driver, 2.5).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))  # Modify selector as needed
    )
    button.click()
except Exception as e:
    pass

time.sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

links = soup.find_all("a", class_="whitespace-nowrap below-420:w-[100px] below-420:truncate text-mlb-blue-link")

# Print all href links
for link in links:
    print('https://www.mlb.com' + link["href"])



https://www.mlb.com/tv/g717770/vff9794bb-c975-4508-b3f4-9673deeaf30a
https://www.mlb.com/tv/g717770/v0e33c608-bccc-4216-a170-87f1ceb8bfe4
https://www.mlb.com/tv/g717770/v62c2171b-950f-40a2-8be4-0f7da9088d1c
https://www.mlb.com/tv/g717770/vfe11f33d-9eb5-41e6-a58a-8ee38cbbf453
https://www.mlb.com/tv/g717784/v34219737-8903-4169-8486-02ca9863dbc8
https://www.mlb.com/tv/g717784/ved4dcb14-f371-48f4-b87e-4d697d3f3060
https://www.mlb.com/tv/g717784/va56737dd-af4a-4198-9cac-1fa7a7c3ee21
https://www.mlb.com/tv/g717784/v34ae9067-4062-447e-9f4d-60635dc137dc
https://www.mlb.com/tv/g717771/v520f8a66-9a17-4ce4-96ac-b8c7d397983b
https://www.mlb.com/tv/g717771/v88967c91-40ab-4143-bd94-ff5eae2b1fd4
https://www.mlb.com/tv/g717771/vd9b51ead-1819-49d9-9a36-9b53d6ab34eb
https://www.mlb.com/tv/g717771/vf3a88059-3d5a-4df5-9087-626c0d4e25a4
https://www.mlb.com/tv/g717767/v3c28d166-89fd-4333-b38a-bd30adfacde2
https://www.mlb.com/tv/g717767/v995f6b6c-195a-4d1d-bd6a-aa0da20e4f9e
https://www.mlb.com/tv/g717767/v8b

In [ ]:


def get_best_broadcasts(url, username, password):
    import subprocess
    import json

    command = [
        'yt-dlp', '-j', '--username', username, '--password', password, url
    ]
    
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    if result.returncode != 0:
        print("yt-dlp error:", result.stderr)
        return None, None

    try:
        video_info = json.loads(result.stdout)
    except json.JSONDecodeError:
        print("Failed to parse yt-dlp output as JSON.")
        return None, None
    
    formats = video_info.get('formats', [])

    home_broadcasts = [f for f in formats if 'Home' in f.get('format_note', '')]
    away_broadcasts = [f for f in formats if 'Away' in f.get('format_note', '')]

    best_home = max(home_broadcasts, key=lambda f: f.get('width', 0)) if home_broadcasts else None
    best_away = max(away_broadcasts, key=lambda f: f.get('width', 0)) if away_broadcasts else None

    home_id = best_home['format_id'] if best_home else None
    away_id = best_away['format_id'] if best_away else None

    return home_id, away_id


# Example usage
url = "https://www.mlb.com/tv/g747185/vfd3b7ded-13e6-4809-97d1-be14a810750f#game=747185,tfs=20240627_194000,game_state=live"
url = 'https://www.mlb.com/tv/g717760/vf3e13133-e27a-44dd-ac52-843439e064c4'
username = "jaredzrks0@gmail.com"
password = "Emiliopagan450"

home_format_code, away_format_code = get_best_broadcasts(url, username, password)
print(f"Best Home Broadcast Format Code: {home_format_code}")
print(f"Best Away Broadcast Format Code: {away_format_code}")


Best Home Broadcast Format Code: Home-TV-en-4160
Best Away Broadcast Format Code: None


In [21]:
import subprocess

def download_video_and_subtitles(url, username, password, format_code, download_video=True):
    # Base yt-dlp command
    command = ['yt-dlp']
  
    # Add options for subtitles
    command.extend([
        '-f', format_code,
        '--write-subs',           # Write subtitles
        '--sub-lang', 'en',       # Use English subtitles
        '--embed-subs',           # Embed subtitles in the video
        '--username', username,
        '--password', password,
    ])

    if download_video == False:
        command.extend(['--skip-download'])
    

    # Add the URL to the command
    command.append(url)
    print(" ".join(command))
    
    # Run the command
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    # Check if the download was successful
    if result.returncode == 0:
        print("Download successful!")
        print(result.stdout)
    else:
        print("Error occurred during download.")
        print(result.stderr)

# Example usage:
url = "https://www.mlb.com/tv/g747185/vfd3b7ded-13e6-4809-97d1-be14a810750f#game=747185,tfs=20240627_194000,game_state=live"
url = 'https://www.mlb.com/tv/g717766/v2991715d-a939-415d-809f-f349eaa5340d'
username = "jaredzrks0@gmail.com"
password = "Emiliopagan450"
format_code = 'Home-TV-en-4160'

# To download subtitles only (without video)
download_video_and_subtitles(url, username, password, format_code, download_video=False)


yt-dlp -f Home-TV-en-4160 --write-subs --sub-lang en --embed-subs --username jaredzrks0@gmail.com --password Emiliopagan450 --skip-download https://www.mlb.com/tv/g717766/v2991715d-a939-415d-809f-f349eaa5340d
Download successful!
[MLBTV] Logging in
[MLBTV] Downloading session ID
[MLBTV] Extracting URL: https://www.mlb.com/tv/g717766/v2991715d-a939-415d-809f-f349eaa5340d
[MLBTV] 717766: Downloading JSON metadata
[MLBTV] 717766: Downloading Home-TV-en broadcast JSON
[MLBTV] 717766: Downloading m3u8 information
[MLBTV] 717766: Downloading Away-TV-en broadcast JSON
[MLBTV] 717766: Downloading Away-AM-en broadcast JSON
[MLBTV] 717766: Downloading Home-AM-en broadcast JSON
[MLBTV] 717766: Downloading Away-AM-es broadcast JSON
[MLBTV] 717766: Downloading Home-AM-es broadcast JSON
[MLBTV] 717766: Downloading Home-AM-es broadcast JSON
[info] 717766: Downloading subtitles: en
[info] 717766: Downloading 1 format(s): Home-TV-en-4160
Deleting existing file 2023-06-14 - New York Yankees @ New York M

In [22]:
import subprocess

def convert_vtt_to_srt(input_file, output_file):
    try:
        subprocess.run(
            ['ffmpeg', '-y', '-i', input_file, '-map', '0:s:0', output_file],
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )
        print(f"Successfully converted {input_file} to {output_file}")
    except subprocess.CalledProcessError as e:
        print("FFmpeg error:", e.stderr.decode())

# Example usage
convert_vtt_to_srt('2023-06-14 - New York Yankees @ New York Mets [717766].en.vtt', 'subtitles.srt')

Successfully converted 2023-06-14 - New York Yankees @ New York Mets [717766].en.vtt to subtitles.srt


In [1]:
with open('subtitles.srt', 'r', encoding='utf-8') as file:
            content = file.read()

FileNotFoundError: [Errno 2] No such file or directory: 'subtitles.srt'

In [24]:
def extract_timestamp_groups(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split on double newlines (empty line separates groups in SRT)
    groups = content.strip().split('\n\n')

    timestamp_groups = []
    for group in groups:
        lines = group.strip().split('\n')
        # Usually first line: index, second line: timestamp, rest: text
        if len(lines) >= 2:
            timestamp_line = lines[1]
            text_lines = lines[2:] if len(lines) > 2 else []
            # Join timestamp + text (skip index line)
            timestamp_groups.append('\n'.join(text_lines))
    
    return timestamp_groups

# Example usage:
timestamp_groups = extract_timestamp_groups('subtitles.srt')

In [25]:
def find_overlap(s1, s2, min_overlap=3):
    """Find the longest overlap between the end of s1 and start of s2."""
    max_overlap = 0
    max_len = min(len(s1), len(s2))
    for length in range(max_len, min_overlap - 1, -1):
        if s1[-length:] == s2[:length]:
            return length
    return 0

def reconstruct_text_with_overlap(fragments):
    if not fragments:
        return ""

    result = fragments[0]
    for current in fragments[1:]:
        overlap_len = find_overlap(result, current)
        # Append only the non-overlapping part of current
        result += current[overlap_len:]
    # Optional: normalize spaces
    result = " ".join(result.split())
    return result



text = reconstruct_text_with_overlap(timestamp_groups)
print(text)



AND DAVIDCONE, KAEL, TAKE IT AWAY. AND DAVIDCONE, KAEL, TAKE IT AWAY. KARL: WELCOME TO CITI FIELD WHERE THE SKIES HAVE CLEARED, THE RAIN HAS PASSED, AND YOU CAN SEE AN OUTSTANDING NIGHT OF BASEBALL. GOOD WEATHER, GOOD TEAMS. THE METS STRUGGLING RIGHT NOW BUT JUSTIN VERLANDER WILL BE ON THE BUMP AND HE AND ASKED SCHERZER HAVE NOT BEEN LEADING THEM WHERE THEY WANT TO. CAN HE TURN IT AROUND AGAINST A FORMER TEAMMATE? AND THE GUY WHO HAS HAD A PHENOMENAL SEASON? GERRIT COLE OF THE NEW YORK YANKEES. IT IS THE SUBWAY SERIES. TWO ACES ON THE MOUND AND THEY HAVE GOTTEN TOGETHER BEFORE. YES, ON THE SAME TEAM. THEY ALSO FACED EACH OTHER. YOU HAVE TO GO BACK TO 2015. REMEMBER THE PITTSBURGH PIRATES. VERLANDER HITS STARLING MARTE IN THAT GAME. IT ENDED UP GOING 14 INNINGS. NEITHER GUY FINISHED AS SOMEBODY INVOLVED IN THE DECISION. THEY EACH WENT SIX. FIVE WALKS FOR VERLANDER IN THAT GAME. THEY GOT TOGETHER IN HOUSTON IN 18 AND 19 AND THAT IS WHERE BRENT STROM CONVINCED THEM, LET‘S DITCH THE TWO SE

In [3]:
import pickle as pkl

with open('../../../data/game_code_map/map.pkl', 'rb') as fpath:
    pkl.load(fpath)